In [ ]:
from hanlp_restful import HanLPClient
import json
import os
import time

HanLP = HanLPClient('https://www.hanlp.com/api', auth="", language='zh')
HanLP.parse(["商品和服务。晓美焰来到北京立方庭参观自然语义科技公司。", "我爸是李刚"], tasks='pos/ctb')

{'tok/fine': [['商品',
   '和',
   '服务',
   '。',
   '晓美焰',
   '来到',
   '北京',
   '立方庭',
   '参观',
   '自然',
   '语义',
   '科技',
   '公司',
   '。'],
  ['我', '爸', '是', '李刚']],
 'pos/ctb': [['NN',
   'CC',
   'NN',
   'PU',
   'NR',
   'VV',
   'NR',
   'NR',
   'VV',
   'NN',
   'NN',
   'NN',
   'NN',
   'PU'],
  ['PN', 'NN', 'VC', 'NR']]}

In [2]:

def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [3]:
block_size = 150
big_block_size = 14000

results_dict = dict()

In [4]:
def join(ll):
    return_l = []
    for l in ll:
        return_l = return_l + l
    return return_l

In [ ]:
texts = []
query_timestamps = []
root = "Chinese"
for file_name in os.listdir(root):
    print("currently processing", file_name)
    if file_name in results_dict:
        continue
    with open(os.path.join(root, file_name), 'r', encoding='utf-8') as rf:
        text = rf.read()
#         def read_in_blocks(text, block_size=15000):
#             while True:
#                 block = text.read(block_size)
#             if not block:
#                 return  # 文件读取完毕
#         read_in_blocks(text)
    
    all_tokens = []
    all_pos_tags = []
    
    big_block_list = split_into_blocks(text, 14000)
    
    for big_block in big_block_list:
        block_list = split_into_blocks(big_block, 150)

        if (len(query_timestamps) < 60):
            gap = 10000
        else:
            current_time = time.time()
            gap = current_time - query_timestamps[-60]
        if gap < 70:
            print("\tsleeping for", 70-gap)
            time.sleep(70-gap)
        query_timestamps.append(time.time())
        
        # 分词
        data = HanLP.parse(block_list, tasks='pos/ctb')
        tokens = data['tok/fine']
        pos_tags = data['pos/ctb']

        all_tokens = all_tokens + join(tokens)
        all_pos_tags = all_pos_tags + join(pos_tags)

    results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}
#     print (all_tokens)

In [ ]:
import json

#with open("results_dict_Chinese.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)

with open("results_dict_Chinese.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [6]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [7]:
def merge_stopwords(*stopwords_lists):
    merged_stopwords = set()

    for stopwords_file in stopwords_lists:
        with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())
            merged_stopwords.update(stopwords)

    return list(merged_stopwords)

stopwords_list1 = 'scu_stopwords.txt'
stopwords_list2 = 'cn_stopwords.txt'
stopwords_list3 = 'baidu_stopwords.txt'
stopwords_list4 = 'hit_stopwords_Ch.txt'
stopwords_list5 = 'cust_stopwords.txt'

merged_stopwords = merge_stopwords(stopwords_list1, stopwords_list2, stopwords_list3, stopwords_list4, stopwords_list5)

In [8]:
for filename in results_dict:
    token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in token_list if word not in merged_stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)

In [9]:
texts = []
numbers = []
textid2name = dict()
for key, item in results_dict.items():
    texts.append(item['cleaned_noun_tokens'])
    textid2name[len(texts)-1] = key
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)

def split_list(l, length=1000):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)
new_texts = []

chunk2text = dict()
for i in range(len(texts)):
    text = texts[i]
    splitted_text = split_list(text)
    for j in range(len(splitted_text)):
        new_texts.append(splitted_text[j])
        chunk2text[len(new_texts)-1] = {"id of original doc": i, "name of original doc": textid2name[i], "chunk index": j, "total # chunks": len(splitted_text)}
print(len(new_texts))
texts = new_texts

[445, 564, 753, 1063, 1208, 1246, 1444, 2568, 2605, 3064, 3472, 4187, 4226, 4362, 4381, 4666, 5153, 5570, 5664, 6301, 6825, 6829, 8129, 9149, 9743, 10808, 10856, 11029, 11040, 11678, 12023, 12418, 14050, 14316, 14324, 14420, 14903, 15542, 15691, 16244, 16760, 17359, 17657, 19511, 22074]
439


In [10]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [11]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [ ]:
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=100,random_state = 100, iterations=300, chunksize = 2200,eval_every = None)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(5,12)
# z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': y,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()

In [12]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=100,random_state = 100, iterations=300, chunksize = 2200,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.021*"女" + 0.018*"王" + 0.014*"兵" + 0.011*"衆" + 0.011*"軍" + 0.008*"城" + 0.007*"親王" + 0.007*"敵" + 0.007*"馬" + 0.006*"衣"'), (1, '0.020*"女主" + 0.015*"國王" + 0.010*"小人" + 0.006*"劍" + 0.006*"國" + 0.004*"臣" + 0.004*"廟" + 0.004*"帝" + 0.004*"雄" + 0.004*"英國"'), (2, '0.015*"物" + 0.010*"鳥" + 0.010*"石" + 0.009*"樹" + 0.008*"水" + 0.008*"力" + 0.007*"洞" + 0.007*"狗" + 0.007*"狀" + 0.007*"妻"'), (3, '0.015*"王" + 0.010*"神" + 0.010*"狀" + 0.008*"女" + 0.008*"女王" + 0.007*"屍" + 0.007*"國" + 0.005*"今日" + 0.005*"目" + 0.005*"衆"'), (4, '0.077*"女" + 0.019*"老人" + 0.017*"金" + 0.015*"馬" + 0.009*"雄" + 0.008*"劍" + 0.008*"父" + 0.006*"鬼" + 0.005*"野人" + 0.005*"洞"'), (5, '0.028*"舟" + 0.013*"船" + 0.009*"書" + 0.007*"狀" + 0.006*"島" + 0.006*"衆" + 0.006*"海" + 0.006*"力" + 0.005*"船主" + 0.005*"水"'), (6, '0.034*"船" + 0.011*"船主" + 0.010*"國" + 0.008*"地方" + 0.008*"馬" + 0.007*"東西" + 0.006*"小船" + 0.005*"事情" + 0.005*"水" + 0.005*"衆人"')]
3
3
3
3
3
2
5
3
5
5
3
2
3
3
3
3
3
3
3
3
3
3
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
3
4
4
4
4
5
5
5
4
4
3
1

In [13]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.115589 -0.010972       1        1  32.196077
5     -0.044526  0.072732       2        1  23.429338
2     -0.104275  0.091770       3        1  18.135907
0     -0.097724 -0.056235       4        1  10.102759
6      0.176601  0.165681       5        1   9.644126
4      0.004213 -0.138137       6        1   4.068212
1      0.181301 -0.124839       7        1   2.423580, topic_info=     Term         Freq        Total Category  logprob  loglift
113     女  3187.000000  3187.000000  Default  30.0000  30.0000
2635    船  2645.000000  2645.000000  Default  29.0000  29.0000
376     舟  3090.000000  3090.000000  Default  28.0000  28.0000
1038    王  2685.000000  2685.000000  Default  27.0000  27.0000
432     金   796.000000   796.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
4145   皇帝    17.402662   457.970382   Topic7  -6.3134   0.4497
1026    火    17.854452   765.802524   Topic7  -6.2878  -0.0387
270     父    17.472176  1302.062153   Topic7  -6.3094  -0.5912
218     書    17.017835  1890.044312   Topic7  -6.3358  -0.9902
482     馬    16.227031  1481.044760   Topic7  -6.3834  -0.7939

[551 rows x 6 columns], token_table=       Topic      Freq Term
term                       
3          1  0.068343   上帝
3          2  0.271853   上帝
3          3  0.329565   上帝
3          4  0.183767   上帝
3          5  0.060749   上帝
...      ...       ...  ...
35792      3  0.964685   黃油
30296      4  0.944677  黑心郎
35725      3  0.964774   龍蝦
5098       3  0.935360    龜
5098       5  0.035975    龜

[1407 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 1, 7, 5, 2])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Assuming vis_data is already prepared
# vis_data = gensimvis.prepare(lda_model, corpus, id2word)

# Save to a file
pyLDAvis.save_html(vis, 'lda_result_Ch1.html')

In [15]:
from collections import defaultdict
import heapq


In [16]:
# Dictionary to store the top 5 contributing documents for each topic
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 20:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
print("Top 5 documents that contribute the most to each topic:")
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    print(f"Topic {topic_id}:")
    for prob, doc_idx in sorted_docs:
        print(f"  Document {doc_idx} with a probability of {prob}")



Top 5 documents that contribute the most to each topic:
Topic 3:
  Document 108 with a probability of 0.999127209186554
  Document 164 with a probability of 0.9991270303726196
  Document 110 with a probability of 0.9991265535354614
  Document 166 with a probability of 0.9991264343261719
  Document 101 with a probability of 0.9991264343261719
  Document 161 with a probability of 0.9991264343261719
  Document 100 with a probability of 0.9991259574890137
  Document 169 with a probability of 0.9991241693496704
  Document 17 with a probability of 0.9991230964660645
  Document 15 with a probability of 0.9991227984428406
  Document 16 with a probability of 0.9991224408149719
  Document 13 with a probability of 0.9991222620010376
  Document 14 with a probability of 0.9991220831871033
  Document 18 with a probability of 0.999121904373169
  Document 268 with a probability of 0.9991179704666138
  Document 267 with a probability of 0.999117910861969
  Document 269 with a probability of 0.999117553

In [17]:
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 50:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    if topic_id ==3:
        
        for prob, doc_idx in sorted_docs:
            print(f"  Document {doc_idx} with a probability of {prob}")
            print(chunk2text[doc_idx])
        



  Document 108 with a probability of 0.9991270899772644
{'id of original doc': 8, 'name of original doc': '埃司兰情侠传.txt', 'chunk index': 8, 'total # chunks': 11}
  Document 164 with a probability of 0.9991270303726196
{'id of original doc': 16, 'name of original doc': '斐洲煙水愁城録.txt', 'chunk index': 11, 'total # chunks': 17}
  Document 110 with a probability of 0.9991265535354614
{'id of original doc': 8, 'name of original doc': '埃司兰情侠传.txt', 'chunk index': 10, 'total # chunks': 11}
  Document 161 with a probability of 0.9991264343261719
{'id of original doc': 16, 'name of original doc': '斐洲煙水愁城録.txt', 'chunk index': 8, 'total # chunks': 17}
  Document 166 with a probability of 0.9991263747215271
{'id of original doc': 16, 'name of original doc': '斐洲煙水愁城録.txt', 'chunk index': 13, 'total # chunks': 17}
  Document 101 with a probability of 0.9991263747215271
{'id of original doc': 8, 'name of original doc': '埃司兰情侠传.txt', 'chunk index': 1, 'total # chunks': 11}
  Document 100 with a probabili

In [18]:
chunk2text[365]


{'id of original doc': 39,
 'name of original doc': '雾中人.txt',
 'chunk index': 14,
 'total # chunks': 23}

In [19]:
def trace_chunk(chunk_id):
    """
    Trace a chunk ID back to its original tokens and document information.

    Parameters:
    - chunk_id (int): The ID of the chunk to trace.

    Returns:
    - dict: A dictionary containing the original tokens and document metadata.
    """
    if chunk_id in chunk2text:
        # Retrieve the metadata for the chunk
        chunk_info = chunk2text[chunk_id]
        original_doc_id = chunk_info["id of original doc"]
        chunk_index = chunk_info["chunk index"]
        total_chunks = chunk_info["total # chunks"]
        doc_name = chunk_info["name of original doc"]

        # Retrieve the tokens for the chunk
        tokens = texts[chunk_id]  # Since texts has been updated to new_texts (chunks)

        # Return a dictionary with the information
        return {
            "chunk_id": chunk_id,
            "original_doc_id": original_doc_id,
            "doc_name": doc_name,
            "chunk_index": chunk_index,
            "total_chunks": total_chunks,
            "tokens": tokens
        }
    else:
        return {"error": "Chunk ID not found."}

# Example usage:
chunk_id_to_trace = 13  # Replace with the chunk ID you want to trace
result = trace_chunk(chunk_id_to_trace)
print(result)


{'chunk_id': 13, 'original_doc_id': 1, 'doc_name': '三千年艳尸记.txt', 'chunk_index': 7, 'total_chunks': 16, 'tokens': ['尼爾河', '新國', '萌芽', '牛年', '屍', '配', '屍', '夥', '希臘', '古國', '希臘', '陽光', '國人', '希臘', '殘喘', '盛', '狀', '夷', '家聲', '狀', '女王', '耶斐伯來', '雅露撒冷', '王', '廟', '廟', '神', '神', '米逐亞', '米遂', '未來', '先', '影響', '天下', '世界', '猶太人', '遺民', '四裔', '雅露撒冷', '斐伯來', '斐律', '王', '廟', '王', '名', '斐律', '廟', '羅馬', '羅馬', '鷹', '裔胄', '猶太', '沙漠', '王', '羅馬', '族', '運', '國徽', '鷹', '鷹', '臘丁語', '天下', '清寂', '沙漠', '承平', '王', '臘丁語', '參差', '文字', '學', '今日', '穌', '博', '世界', '中學', '希臘語', '斐伯來', '文字', '世界', '文字', '王', '樹', '花', '善果', '猶太', '異教', '哲學', '卑賤', '衆', '十字架', '賢', '炳若日星', '賢', '上帝', '子', '世界', '王', '世人', '殘忍', '狼', '宗', '神教', '世界', '今日', '黑面', '米遂亞', '辜', '橫', '理', '果', '上天', '神', '子', '理', '愚', '威權', '教', '力', '空言', '衆', '衆', '輩', '天帝', '耶和華', '巴爾', '阿司拖勒', '外道', '米遂', '出身', '天地', '記性', '劣', '先知', '輩', '收局', '屍', '豔', '屍', '猶太', '天下', '國', '先', '古族人', '當日', '雅露撒冷', '輩', '石', '大廟', '教', '老人', '鬣', '惡', '百姓', '石', '紗'

In [20]:
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 50:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    if topic_id ==4:
        
        for prob, doc_idx in sorted_docs:
            print(f"  Document {doc_idx} with a probability of {prob}")
            print(chunk2text[doc_idx])

  Document 438 with a probability of 0.8924302458763123
{'id of original doc': 44, 'name of original doc': '鲁滨逊漂流记(林纾).txt', 'chunk index': 18, 'total # chunks': 19}
  Document 54 with a probability of 0.8568844795227051
{'id of original doc': 4, 'name of original doc': '双雄较剑录.txt', 'chunk index': 15, 'total # chunks': 16}
  Document 76 with a probability of 0.7591680884361267
{'id of original doc': 6, 'name of original doc': '古鬼遺金記.txt', 'chunk index': 4, 'total # chunks': 11}
  Document 42 with a probability of 0.7225961089134216
{'id of original doc': 4, 'name of original doc': '双雄较剑录.txt', 'chunk index': 3, 'total # chunks': 16}
  Document 78 with a probability of 0.718842089176178
{'id of original doc': 6, 'name of original doc': '古鬼遺金記.txt', 'chunk index': 6, 'total # chunks': 11}
  Document 48 with a probability of 0.6913259625434875
{'id of original doc': 4, 'name of original doc': '双雄较剑录.txt', 'chunk index': 9, 'total # chunks': 16}
  Document 79 with a probability of 0.684294

In [21]:
# Specify the document index you want to analyze
doc_idx = 13  # Replace with the desired document index

# Get the topic probabilities for the specified document
doc_topics = lda.get_document_topics(corpus[doc_idx])

# Print the contributions of the specified document to different topics
print(f"Topic contributions for document {doc_idx}:")
for topic_id, prob in doc_topics:
    print(f"  Topic {topic_id}: Probability {prob}")

Topic contributions for document 13:
  Topic 3: Probability 0.9991222023963928


In [22]:
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 50:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    if topic_id ==0:
        
        for prob, doc_idx in sorted_docs:
            print(f"  Document {doc_idx} with a probability of {prob}")
            print(chunk2text[doc_idx])

  Document 151 with a probability of 0.9991233348846436
{'id of original doc': 15, 'name of original doc': '撷兰记.txt', 'chunk index': 16, 'total # chunks': 18}
  Document 116 with a probability of 0.9990734457969666
{'id of original doc': 9, 'name of original doc': '天女離魂記.txt', 'chunk index': 5, 'total # chunks': 7}
  Document 144 with a probability of 0.9545880556106567
{'id of original doc': 15, 'name of original doc': '撷兰记.txt', 'chunk index': 9, 'total # chunks': 18}
  Document 114 with a probability of 0.9416607022285461
{'id of original doc': 9, 'name of original doc': '天女離魂記.txt', 'chunk index': 3, 'total # chunks': 7}
  Document 145 with a probability of 0.9004166126251221
{'id of original doc': 15, 'name of original doc': '撷兰记.txt', 'chunk index': 10, 'total # chunks': 18}
  Document 113 with a probability of 0.8796856999397278
{'id of original doc': 9, 'name of original doc': '天女離魂記.txt', 'chunk index': 2, 'total # chunks': 7}
  Document 327 with a probability of 0.87869042158

In [23]:
def trace_chunk(chunk_id):
    """
    Trace a chunk ID back to its original tokens and document information.

    Parameters:
    - chunk_id (int): The ID of the chunk to trace.

    Returns:
    - dict: A dictionary containing the original tokens and document metadata.
    """
    if chunk_id in chunk2text:
        # Retrieve the metadata for the chunk
        chunk_info = chunk2text[chunk_id]
        original_doc_id = chunk_info["id of original doc"]
        chunk_index = chunk_info["chunk index"]
        total_chunks = chunk_info["total # chunks"]
        doc_name = chunk_info["name of original doc"]

        # Retrieve the tokens for the chunk
        tokens = texts[chunk_id]  # Since texts has been updated to new_texts (chunks)

        # Return a dictionary with the information
        return {
            "chunk_id": chunk_id,
            "original_doc_id": original_doc_id,
            "doc_name": doc_name,
            "chunk_index": chunk_index,
            "total_chunks": total_chunks,
            "tokens": tokens
        }
    else:
        return {"error": "Chunk ID not found."}

# Example usage:
chunk_id_to_trace = 167 # Replace with the chunk ID you want to trace
result = trace_chunk(chunk_id_to_trace)
print(result)


{'chunk_id': 167, 'original_doc_id': 16, 'doc_name': '斐洲煙水愁城録.txt', 'chunk_index': 14, 'total_chunks': 17, 'tokens': ['車', '馬', '晏豐', '副', '車', '書', '狀', '始', '侏儒', '仇', '奴', '王度', '民', '國', '外寶', '學', '民', '狼', '學', '侶', '貴族', '天壤', '國民', '民', '阿森尼亞', '族', '人心', '輩', '儀', '觀', '美', '國', '俗', '儀', '觀', '衆', '偉', '神', '國', '父老', '婦孺', '風貌', '吾國', '魁偉', '匹', '美', '配', '佳偶', '陽神', '英', '偶', '女王', '輿論', '戰端', '兵事', '國', '烽火', '天', '妖', '讖', '外人', '偶', '百姓', '陽神', '敵', '白', '幸', '爲人', '陽', '國', '衆', '令', '名', '小人', '異國', '辱', '夜', '婚禮', '明日', '檄', '甲士', '蟻', '國', '洲', '煙', '水', '鶴', '議員', '將領', '勳', '臣', '兵事', '部署', '防守', '庶務', '兵', '苑', '美路斯', '沿路', '纛', '帳', '步兵', '馬隊', '兵力', '麗士', '所部', '諜', '軍', '魔士', '兵', '中軍', '阿司達那', '美路斯', '界', '小侯', '兵', '朔方', '勁旅', '山', '兵', '戰', '公畢', '路', '馬地', '馬', '軍', '敵', '敵軍', '諜', '營', '臨愁城', '村莊', '僉', '美路斯', '敵', '軍', '軍', '敵', '敵', '血', '壯士', '僉', '主', '衆', '毫', '豬', '穴旦', '火', '氣勢', '趣', '地圖', '阿司達那', '遠', '大軍', '壁', '壁', '美路斯', '山路', '眯', '旁', '樹叢', '